In [1]:
#from veneer.manage import create_command_line

#import veneer-py

from veneer.manage import start, create_command_line, kill_all_now
import veneer
import geopandas
import pandas as pd

nodeOfInterestName = '120212A'
outputCSV = 'D:/aa/testingSubCats.csv'

In [2]:
veneer_install = 'D:/PythonScriptingForGBR/VeneerPlugin/v411'
source_version = '4.1.1'
cmd_directory = 'C:\\tempveneer\\veneer_cmd'
path = create_command_line(veneer_install,source_version,dest=cmd_directory)
path

'C:\\tempveneer\\veneer_cmd\\FlowMatters.Source.VeneerCmd.exe'

In [3]:
catchment_project='D:\\Xike\\bu\\bu_reefhydro_bo_411_noMP.rsproj'
num_copies=1    # Important - set this to be a number ~ the number of CPU cores in your system!
first_port=9950

In [4]:
processes,ports = start(catchment_project,
                        n_instances=num_copies,
                        ports=first_port,
                        debug=True,
                        veneer_exe=path,
                        remote=False)

Starting C:\tempveneer\veneer_cmd\FlowMatters.Source.VeneerCmd.exe -p 9950 -s  "D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj"
[0] Loading plugins

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\DERMTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\PestFileProductionTools.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\RebuildInternalSubcatchmentRaster.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\SneakyLinkModelEditor.dll

[0] Loaded C:\Program Files\eWater\Source 4.1.1.4513\Plugins\ReefHydroCalModels.dll

[0] Plugins loaded (5/5)

[0] Opening project file: D:\Xike\bu\bu_reefhydro_bo_411_noMP.rsproj

ERROR[0] log4net:ERROR Failed to find configuration section 'log4net' in the application's .config file. Check your .config file for the <log4net> and <configSections> elements. The configuration section should look like: <section name="log4net" type="log4net.Config.Log4NetConfigurationSectionHandler,log4net" />

[0] L

In [5]:
vs = [veneer.Veneer(port=p) for p in ports]

In [6]:
#kill_all_now(processes)

In [7]:
#%matplotlib inline
v = vs[0]
#v.network().as_dataframe().plot()

In [8]:
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/01/2000 12:00:00 AM
31/03/2000 11:00:00 PM


In [9]:
the_network = v.network()
#outlets = the_network.outlet_nodes()
#outlets

In [10]:
def processLinks(theNetwork, currentLink):
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    #print("Now processing: " + theLinkName)
          
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    #print(thisCatName)
    #thisCatArea = theCat['properties']['areaInSquareMeters']
    
    GlobalCatchList.append(thisCatName)
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
        
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, upLink)

    return

In [11]:
GlobalCatchList = []

nodeOfInterest = the_network['features'].find_by_name(nodeOfInterestName)
#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(nodeOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)


In [12]:
contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
contCatchsDF


,SubCats
0,SC #153
1,SC #154


In [13]:
contCatchsDF.to_csv(outputCSV, index=False)

In [14]:
kill_all_now(processes)